# Strands Agent SDK

In [2]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

In [5]:
import os
from strands import Agent
from strands.models import BedrockModel
from src.utils.bedrock import bedrock_info
from botocore.config import Config

## 1. Utils

### 1.1 Get llm model by inference type

In [6]:
def get_model(**kwargs):

    llm_type = kwargs["llm_type"]
    cache_type = kwargs["cache_type"]
    enable_reasoning = kwargs["enable_reasoning"]

    if llm_type in ["claude-sonnet-3-7", "claude-sonnet-4"]:
        
        if llm_type == "claude-sonnet-3-7": model_name = "Claude-V3-7-Sonnet-CRI"
        elif llm_type == "claude-sonnet-4": model_name = "Claude-V4-Sonnet-CRI"

        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name=model_name),
            streaming=True,
            max_tokens=8192*5,
            stop_sequences=["\n\nHuman"],
            temperature=1 if enable_reasoning else 0.01, 
            additional_request_fields={
                "thinking": {
                    "type": "enabled" if enable_reasoning else "disabled", 
                    **({"budget_tokens": 8192} if enable_reasoning else {}),
                }
            },
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="adaptive"),
            )
        )   
    elif llm_type == "claude-sonnet-3-5-v-2":
        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192,
            stop_sequences=["\n\nHuman"],
            temperature=0.01,
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="standard"),
            )
        )
    else:
        raise ValueError(f"Unknown LLM type: {llm_type}")

    return llm

### 1.2 Create agent

In [7]:
from datetime import datetime
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [8]:
class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def apply_prompt_template(prompt_name: str, prompt_context={}) -> str:
    
    system_prompts = open(os.path.join("./prompts", f"{prompt_name}.md")).read()    
    #system_prompts = open(os.path.join(os.path.dirname(__file__), f"{prompt_name}.md")).read()
    context = {"CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z")}
    context.update(prompt_context)
    system_prompts = system_prompts.format(**context)
        
    return system_prompts

class ConversationEditor(SlidingWindowConversationManager):

    """
    Manager that only operates on overflow.

        Args:
            window_size (int, optional): Maximum number of messages to retain when
                context overflow occurs. Defaults to 20.
            should_truncate_results (bool, optional): If True, truncate large tool
                results with a placeholder message when overflow happens. If False,
                preserve full tool results but remove more historical messages.
                Defaults to True.
    """

    def __init__(self, window_size=7, should_truncate_results=False):
        super().__init__(
            window_size=window_size,
            should_truncate_results=should_truncate_results
        )

    def apply_management(self, agent, **kwargs):
        """After each event loop - do nothing"""
        print("None")
        pass

    def reduce_context(self, agent, e=None, **kwargs):
        """Only on overflow - use parent class's reduce_context"""
        print(f"⚠️ Overflow occurred! Cleaning up {len(agent.messages)} messages...")

        # 부모 클래스의 reduce_context는 should_truncate_results를 자동으로 처리
        super().reduce_context(agent, e, **kwargs)

        print(f"✅ Cleanup complete: {len(agent.messages)} messages remaining")

In [9]:
def get_agent(**kwargs):

    agent_name, system_prompts = kwargs["agent_name"], kwargs["system_prompts"]
    agent_type = kwargs.get("agent_type", "claude-sonnet-3-7")
    enable_reasoning = kwargs.get("enable_reasoning", False)
    prompt_cache_info = kwargs.get("prompt_cache_info", (False, None)) # (True, "default")
    tools = kwargs.get("tools", None)
    streaming = kwargs.get("streaming", True)
    
    context_overflow_window_size = kwargs.get("context_overflow_window_size", 15)
    context_overflow_should_truncate_results = kwargs.get("context_overflow_should_truncate_results", False)

    prompt_cache, cache_type = prompt_cache_info
    if prompt_cache: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
    else: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

    llm = get_model(llm_type=agent_type, cache_type=cache_type, enable_reasoning=enable_reasoning)
    llm.config["streaming"] = streaming

    agent = Agent(
        model=llm,
        system_prompt=system_prompts,
        tools=tools,
        conversation_manager=ConversationEditor(
            window_size=context_overflow_window_size,
            should_truncate_results=context_overflow_should_truncate_results
        ),
        callback_handler=None # async iterator로 대체 하기 때문에 None 설정
    )

    return agent

### 1.3 Response with streaming

In [10]:
import traceback
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [11]:
class ColoredStreamingCallback(StreamingStdOutCallbackHandler):
    COLORS = {
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'red': '\033[91m',
        'purple': '\033[95m',
        'cyan': '\033[96m',
        'white': '\033[97m',
    }
    
    def __init__(self, color='blue'):
        super().__init__()
        self.color_code = self.COLORS.get(color, '\033[94m')
        self.reset_code = '\033[0m'
    
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{self.color_code}{token}{self.reset_code}", end="", flush=True)

In [12]:
async def process_streaming_response(agent, message):
    callback_reasoning, callback_answer = ColoredStreamingCallback('purple'), ColoredStreamingCallback('white')
    response = {"text": "","reasoning": "", "signature": "", "tool_use": None, "cycle": 0}
    try:
        agent_stream = agent.stream_async(message)
        async for event in agent_stream:
            if "reasoningText" in event:
                response["reasoning"] += event["reasoningText"]
                callback_reasoning.on_llm_new_token(event["reasoningText"])
            elif "reasoning_signature" in event:
                response["signature"] += event["reasoning_signature"]
            elif "data" in event:
                response["text"] += event["data"]
                callback_answer.on_llm_new_token(event["data"])
            elif "current_tool_use" in event and event["current_tool_use"].get("name"):
                response["tool_use"] = event["current_tool_use"]["name"]
                if "event_loop_metrics" in event:
                    if response["cycle"] != event["event_loop_metrics"].cycle_count:
                        response["cycle"] = event["event_loop_metrics"].cycle_count
                        callback_answer.on_llm_new_token(f' \n## Calling tool: {event["current_tool_use"]["name"]} - # Cycle: {event["event_loop_metrics"].cycle_count}\n')
    except Exception as e:
        print(f"Error in streaming response: {e}")
        print(traceback.format_exc())  # Detailed error logging
    
    return agent, response

## 2. Usage

### 2.1 Agent definition

- system prompt

In [13]:
%%writefile ./prompts/task_agent.md
---
CURRENT_TIME: {CURRENT_TIME}
---

You are Bedrock-Manus, a friendly AI assistant developed by AWS AIML Specialist SA Dongjin Jang.
You specialize in handling greetings, small talk, and knowledge-based question answering using available tools.

## Available Tools

You have access to the following tools that you should use when appropriate:

### 1. RAG Tool (rag_tool)
**When to use**: Use this tool when users ask questions that require information from a knowledge base or document collection. This includes:
- Questions about specific topics that might be documented
- Requests for factual information that could be in indexed documents
- Queries about policies, procedures, or technical documentation
- Any question where you need to retrieve and reference specific information

**What it does**: Performs Retrieval-Augmented Generation (RAG) by searching through indexed documents in OpenSearch and generating contextual answers based on retrieved information.

**Input**: A query string containing the user's question

**Example scenarios**:
- "What is the investment return rate for maturity repayment?"
- "Can you explain the company's vacation policy?"
- "How does the authentication system work?"

### 2. Python REPL Tool (python_repl_tool)
**When to use**: Use this tool when users need to execute Python code or perform data analysis:
- Running Python scripts or code snippets
- Data analysis and calculations
- Testing code functionality
- Mathematical computations

**What it does**: Executes Python code in a REPL environment and returns the output

**Input**: Python code string

### 3. Bash Tool (bash_tool) 
**When to use**: Use this tool when users need to execute system commands or perform file operations:
- Running shell commands
- File system operations (ls, mkdir, etc.)
- System information queries
- Development tasks requiring command line operations

**What it does**: Executes bash commands and returns the output

**Input**: A bash command string

## Tool Usage Guidelines

1. **Assess the user's request** - Determine if the question requires tool usage
2. **Choose the appropriate tool** - Select based on the type of information needed
3. **Use RAG tool for knowledge queries** - When the user asks about topics that might be in your knowledge base
4. **Use Python REPL for code execution** - When the user needs to run Python code or perform calculations
5. **Use Bash tool for system operations** - When the user needs to interact with the system
6. **Provide helpful responses** - Always explain the results in a user-friendly way

## Response Style

- Be friendly and conversational
- Provide clear, helpful answers
- When using tools, explain what you're doing and why
- If a tool doesn't provide the needed information, acknowledge this and offer alternatives
- Always prioritize user experience and clarity

Remember to use tools proactively when they can help answer user questions more accurately or completely.

Overwriting ./prompts/task_agent.md


In [14]:
agent = get_agent(
    agent_name="task_agent",
    system_prompts=apply_prompt_template(prompt_name="task_agent", prompt_context={}),
    agent_type="claude-sonnet-4",  # claude-sonnet-3-7, claude-sonnet-4
    prompt_cache_info=(True, "default"),  # enable prompt caching for reasoning agent
    streaming=True,
)

TASK_AGENT - Prompt Cache Enabled


### 2.2 Execution

In [15]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [16]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))


안녕하세요 장동진님! 반갑습니다! 😊

저는 Bedrock-Manus입니다. AWS AIML Specialist SA인 장동진님이 개발해주신 AI 어시스턴트예요. 

오늘 어떤 도움이 필요하신가요? 궁금한 것이 있으시거나 대화하고 싶은 주제가 있으시면 언제든 말씀해 주세요!None


## 3. Tools

In [17]:
from src.tools import python_repl_tool, bash_tool, rag_tool

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [18]:
agent = get_agent(
    agent_name="task_agent",
    system_prompts=apply_prompt_template(prompt_name="task_agent", prompt_context={}),
    agent_type="claude-sonnet-4",  # claude-sonnet-3-7, claude-sonnet-4
    prompt_cache_info=(True, "default"),  # enable prompt caching for reasoning agent
    enable_reasoning=True,
    streaming=True,
    tools=[python_repl_tool, bash_tool, rag_tool],
)

TASK_AGENT - Prompt Cache Enabled


In [19]:
message = "./prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?"
agent, response = asyncio.run(process_streaming_response(agent, message))

The user is asking to check what files are in the "./prompts" directory in Korean. This is a request to check the contents of a directory, which is a file system operation. I should use the bash_tool to execute an `ls` command to list the contents of the ./prompts directory../prompts 디렉토리의 파일 목록을 확인해드리겠습니다.


INFO [src.tools.bash_tool] ===== Executing Bash =====

INFO [src.tools.bash_tool] ===== Coder - Command: ls -la ./prompts =====

INFO [src.tools.decorators] 
Tool - handle_bash_tool returned:
ls -la ./prompts||total 12
drwxrwxr-x 2 ubuntu ubuntu 4096 Oct 14 09:25 .
drwxrwxr-x 3 ubuntu ubuntu 4096 Oct 14 09:25 ..
-rw-rw-r-- 1 ubuntu ubuntu 2959 Oct 25 08:48 task_agent.md




./prompts 디렉토리에는 다음 파일이 있습니다:

- **task_agent.md** (2,959 bytes, 2025년 10월 25일 오전 8시 48분에 마지막으로 수정됨)

이 디렉토리에는 현재 하나의 마크다운 파일만 포함되어 있네요. task_agent.md 파일의 내용을 확인하고 싶으시면 알려주세요!None


In [20]:
message = "Hello world 를 프린팅하는 파이썬 코드를 작성하고 실행시켜 줄래?"
agent, response = asyncio.run(process_streaming_response(agent, message))

사용자가 "Hello world"를 프린팅하는 파이썬 코드를 작성하고 실행해달라고 요청했습니다. 이는 python_repl_tool을 사용해야 하는 상황입니다.

간단한 print("Hello world") 코드를 작성하고 실행하면 됩니다."Hello world"를 출력하는 파이썬 코드를 작성하고 실행해드리겠습니다!


INFO [src.tools.python_repl_tool] ===== Executing Python code =====

INFO [src.tools.python_repl_tool] ===== Code execution successful =====

INFO [src.tools.decorators] Tool - Successfully executed:

```python
print("Hello world")
```


INFO [src.tools.decorators] 
Stdout: Hello world



완료되었습니다! 간단한 파이썬 코드로 "Hello world"를 성공적으로 출력했습니다. 

```python
print("Hello world")
```

이 코드가 실행되어 "Hello world"가 출력되었네요. 다른 파이썬 코드를 실행해보고 싶으시면 언제든지 말씀해주세요!None


In [21]:
#message = "만기 상환에 따른 수익률을 알려줄래?"
message = "만기 상환에 따른 수익률을 알려줄래?, 툴 결과를 받아서 정리하지말고 '완료' 라고만 말해줘"
agent, response = asyncio.run(process_streaming_response(agent, message))

사용자가 "만기 상환에 따른 수익률"에 대해 질문하고 있습니다. 이는 금융이나 투자와 관련된 정보로 보이며, 지식 베이스에서 찾을 수 있는 정보일 가능성이 높습니다. RAG 도구를 사용해야겠네요.

그리고 사용자가 명시적으로 "툴 결과를 받아서 정리하지말고 '완료' 라고만 말해줘"라고 했으므로, 툴 실행 후 결과를 정리하지 않고 단순히 "완료"라고만 응답해야 합니다.


INFO [src.tools.rag_tool] ===== Executing RAG =====

INFO [src.tools.rag_tool] ===== RAG - Query: 만기 상환에 따른 수익률 =====


boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
Bedrock Embeddings Model Loaded
verbose False
이 파생결합증권(DLS)의 만기 상환에 따른 수익률은 기초자산의 만기평가가격에 따라 다음과 같이 결정됩니다:

1. 만기평가가격이 최초기준가격의 100% 이상인 경우:
   - 총액면금액 × [100% + {(만기평가가격-최초기준가격)/최초기준가격 × 70%}]
   - 이때 만기수익률이 낮은 기초자산을 기준으로 계산합니다.
   - 즉, 기초자산(KOSPI200 지수, 금가격지수)의 상승률에 70%를 곱한 수익률이 보장됩니다.

2. 만기평가가격이 최초기준가격의 100% 미만인 경우:
   - 총액면금액 × 100%
   - 기초자산의 하락에도 원금은 100% 보장됩니다.

만기수익률 계산 공식: (만기평가가격-최초기준가격)/최초기준가격
계산 시 (만기평가가격-최초기준가격)/최초기준가격×70%는 소수점 다섯째자리 이하 절사됩니다.

이 상품은 원금보장형으로, 어떤 상황에서도 투자원금은 보장받을 수 있습니다.


INFO [src.tools.decorators] 
Tool - handle_rag_tool returned:
만기 상환에 따른 수익률||이 파생결합증권(DLS)의 만기 상환에 따른 수익률은 기초자산의 만기평가가격에 따라 다음과 같이 결정됩니다:

1. 만기평가가격이 최초기준가격의 100% 이상인 경우:
   - 총액면금액 × [100% + {(만기평가가격-최초기준가격)/최초기준가격 × 70%}]
   - 이때 만기수익률이 낮은 기초자산을 기준으로 계산합니다.
   - 즉, 기초자산(KOSPI200 지수, 금가격지수)의 상승률에 70%를 곱한 수익률이 보장됩니다.

2. 만기평가가격이 최초기준가격의 100% 미만인 경우:
   - 총액면금액 × 100%
   - 기초자산의 하락에도 원금은 100% 보장됩니다.

만기수익률 계산 공식: (만기평가가격-최초기준가격)/최초기준가격
계산 시 (만기평가가격-최초기준가격)/최초기준가격×70%는 소수점 다섯째자리 이하 절사됩니다.

이 상품은 원금보장형으로, 어떤 상황에서도 투자원금은 보장받을 수 있습니다.



완료None


## 4. built-in utility

In [22]:
from pprint import pprint

### 4.1 Check agent

- Syetem prompt

In [23]:
system_prompt = agent.system_prompt
pprint(system_prompt)

('---\n'
 'CURRENT_TIME: Fri Oct 24 2025 22:59:28 \n'
 '---\n'
 '\n'
 'You are Bedrock-Manus, a friendly AI assistant developed by AWS AIML '
 'Specialist SA Dongjin Jang.\n'
 'You specialize in handling greetings, small talk, and knowledge-based '
 'question answering using available tools.\n'
 '\n'
 '## Available Tools\n'
 '\n'
 'You have access to the following tools that you should use when '
 'appropriate:\n'
 '\n'
 '### 1. RAG Tool (rag_tool)\n'
 '**When to use**: Use this tool when users ask questions that require '
 'information from a knowledge base or document collection. This includes:\n'
 '- Questions about specific topics that might be documented\n'
 '- Requests for factual information that could be in indexed documents\n'
 '- Queries about policies, procedures, or technical documentation\n'
 '- Any question where you need to retrieve and reference specific '
 'information\n'
 '\n'
 '**What it does**: Performs Retrieval-Augmented Generation (RAG) by searching '
 'through i

- Message history

In [24]:
agent_messages = agent.messages
pprint(agent_messages)

[{'content': [{'text': './prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?'}], 'role': 'user'},
 {'content': [{'text': './prompts 디렉토리에 있는 파일들을 확인해드리겠습니다.'},
              {'toolUse': {'input': {'cmd': 'ls -la ./prompts'},
                           'name': 'bash_tool',
                           'toolUseId': 'tooluse_46pu4R27Q5CF-BzXpEq3XA'}}],
  'role': 'assistant'},
 {'content': [{'toolResult': {'content': [{'text': 'ls -la ./prompts||total '
                                                   '12\n'
                                                   'drwxrwxr-x 2 ubuntu ubuntu '
                                                   '4096 Oct 14 09:25 .\n'
                                                   'drwxrwxr-x 3 ubuntu ubuntu '
                                                   '4096 Oct 14 09:25 ..\n'
                                                   '-rw-rw-r-- 1 ubuntu ubuntu '
                                                   '2959 Oct 24 22:58 '
                                          

- observility

In [25]:
pprint(agent.event_loop_metrics)

EventLoopMetrics(cycle_count=4,
                 tool_metrics={'bash_tool': ToolMetrics(tool={'input': {'cmd': 'ls '
                                                                               '-la '
                                                                               './prompts'},
                                                              'name': 'bash_tool',
                                                              'toolUseId': 'tooluse_46pu4R27Q5CF-BzXpEq3XA'},
                                                        call_count=1,
                                                        success_count=1,
                                                        error_count=0,
                                                        total_time=0.005445718765258789),
                               'rag_tool': ToolMetrics(tool={'input': {'query': '만기 '
                                                                                '상환에 '
                                  

- Resume

In [ ]:
llm_ = BedrockModel(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
    streaming=True,
    max_tokens=8192,
    stop_sequencesb=["\n\nHuman"],
    temperature=0.01,
    cache_prompt=None, # None/ephemeral/defalut
    #cache_tools: Cache point type for tools
    boto_client_config=Config(
        read_timeout=900,
        connect_timeout=900,
        retries=dict(max_attempts=50, mode="standard"),
    )
)


agent_ = Agent(
    model=llm_,
    tools=[python_repl_tool, bash_tool],
    system_prompt=system_prompt,
    messages=agent_messages,
    callback_handler=None # async iterator로 대체 하기 때문에 None 설정
)

In [ ]:
message = "이어서 대화 하는거 맞니?"
agent, response = asyncio.run(process_streaming_response(agent_, message))

### 4.2 [Conversation management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/?h=conversa)

As conversations grow, managing this context becomes increasingly important for several reasons:

- **Token Limits**: Language models have fixed context windows (maximum tokens they can process)
- **Performance**: Larger contexts require more processing time and resources
- **Relevance**: Older messages may become less relevant to the current conversation
- **Coherence**: Maintaining logical flow and preserving important information


#### 4.2.1. SlidingWindowConversationManager
고정된 수의 최근 메시지를 유지하는 슬라이딩 윈도우 전략을 구현합니다. Agent 클래스에서 기본적으로 사용하는 대화 매니저입니다.

In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [ ]:
# Create a conversation manager with custom window size
conversation_manager = SlidingWindowConversationManager(
    window_size=3,  # Maximum number of messages to keep
    should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window 
)

In [ ]:
agent.conversation_manager = conversation_manager

In [ ]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)

#### 3.1.2. SummarizingConversationManager

오래된 메시지를 요약하여 중요한 정보를 보존하면서 컨텍스트 한계 내에서 대화를 관리합니다.

**주요 설정:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|--------|------|
| `summary_ratio` | `float` | `0.3` | 컨텍스트 축소 시 요약할 메시지 비율 (0.1~0.8 범위) |
| `preserve_recent_messages` | `int` | `10` | 항상 유지할 최근 메시지 수 |
| `summarization_agent` | `Agent` | `None` | 요약 생성용 커스텀 에이전트 (system_prompt와 동시 사용 불가) |
| `summarization_system_prompt` | `str` | `None` | 요약용 커스텀 시스템 프롬프트 (agent와 동시 사용 불가) |

> **기본 요약 방식**: 커스텀 설정이 없을 경우, 주요 토픽, 사용된 도구, 기술적 정보를 3인칭 형태의 구조화된 불릿 포인트로 요약합니다.

In [ ]:
from strands.agent.conversation_manager import SummarizingConversationManager

In [ ]:
# Custom system prompt for technical conversations
custom_system_prompt = """
You are summarizing a technical conversation. Create a concise bullet-point summary that:
- Focuses on code changes, architectural decisions, and technical solutions
- Preserves specific function names, file paths, and configuration details
- Omits conversational elements and focuses on actionable information
- Uses technical terminology appropriate for software development

Format as bullet points without conversational language.
"""

conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=3,  # Always keep 10 most recent messages
    summarization_system_prompt=custom_system_prompt
)

In [ ]:
agent.conversation_manager = conversation_manager

In [ ]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)